# Import Modules

In [1]:
import torch
import torchvision

from time import time
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

C:\Users\62821\.conda\envs\dl_with_me\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
batch_size = 128
crop_size = 64

train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=crop_size, scale=(0.8, 1.0)),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(70),
    transforms.CenterCrop(size=crop_size),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder("../datasets/hidrangea/train/", transform=train_transform)
train_dataloader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

test_set = datasets.ImageFolder("../datasets/hidrangea/test/", transform=test_transform)
test_dataloader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Build Model

## Architecture

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=1024, out_features=256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, 2),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.fc(x)
        
        return x
    
model = Model()

## Loss Function

In [4]:
nll_loss = nn.NLLLoss()
nll_loss

NLLLoss()

## Evaluation Metric

In [5]:
accuracy = Accuracy()
accuracy

Accuracy()

## Optimizer

In [6]:
adamw_optimizer = optim.AdamW(model.parameters(), lr=0.001)
adamw_optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

## Loop Function

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, eval_metric):
    print("Train:")
    
    losses = []
    for batch, (feature, label) in enumerate(dataloader):
        # Forwardpropagation
        pred_label = model(feature)
        loss = loss_fn(pred_label, label)
        acc = eval_metric(pred_label, label)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss = loss.item()
        losses.append(loss)
        
        print(f"Batch-{batch + 1} | Accuracy: {acc:>7f} | Loss: {loss:>7f}")
        
    acc = eval_metric.compute()
    print(f"Accuracy : {acc}")
    print(f"Loss     : {sum(losses) / len(dataloader)}")
    
def test_loop(dataloader, model, loss_fn, eval_metric):
    print("\nTest:")
    
    losses = []
    for batch, (feature, label) in enumerate(dataloader):
        # Forwardpropagation
        pred_label = model(feature)
        loss = loss_fn(pred_label, label)
        acc = eval_metric(pred_label, label)
        
        with torch.no_grad():
            loss = loss.item()
            losses.append(loss)
        
        print(f"Batch-{batch + 1} | Accuracy: {acc:>7f} | Loss: {loss:>7f}")
        
    acc = eval_metric.compute()
    print(f"Accuracy : {acc}")
    print(f"Loss     : {sum(losses) / len(dataloader)}")

In [8]:
epochs = 20

start = time()
for epoch in range(epochs):
    print(f"EPOCH {epoch + 1}")
    print("=" * 46, end="\n")
    train_loop(train_dataloader, model, nll_loss, adamw_optimizer, accuracy)
    test_loop(test_dataloader, model, nll_loss, accuracy)
    print("=" * 46, end="\n\n")
stop = time()

total_time = stop - start
print(f"Training time: {(total_time / 60):.3f} seconds")

EPOCH 1
Train:
Batch-1 | Accuracy: 0.515625 | Loss: 0.693050
Batch-2 | Accuracy: 0.492188 | Loss: 0.698659
Batch-3 | Accuracy: 0.460938 | Loss: 0.700733
Batch-4 | Accuracy: 0.523438 | Loss: 0.691923
Batch-5 | Accuracy: 0.492188 | Loss: 0.693484
Batch-6 | Accuracy: 0.531250 | Loss: 0.691498
Batch-7 | Accuracy: 0.546875 | Loss: 0.692149
Batch-8 | Accuracy: 0.692308 | Loss: 0.690710
Accuracy : 0.527999997138977
Loss     : 0.6940258666872978

Test:
Batch-1 | Accuracy: 0.437500 | Loss: 0.691726
Batch-2 | Accuracy: 0.578125 | Loss: 0.687889
Batch-3 | Accuracy: 0.500000 | Loss: 0.690815
Batch-4 | Accuracy: 0.687500 | Loss: 0.682118
Accuracy : 0.5235714316368103
Loss     : 0.6881367713212967

EPOCH 2
Train:
Batch-1 | Accuracy: 0.507812 | Loss: 0.690732
Batch-2 | Accuracy: 0.554688 | Loss: 0.688826
Batch-3 | Accuracy: 0.468750 | Loss: 0.689673
Batch-4 | Accuracy: 0.609375 | Loss: 0.684494
Batch-5 | Accuracy: 0.656250 | Loss: 0.682227
Batch-6 | Accuracy: 0.578125 | Loss: 0.682501
Batch-7 | Accur

Batch-7 | Accuracy: 0.867188 | Loss: 0.317668
Batch-8 | Accuracy: 0.875000 | Loss: 0.357500
Accuracy : 0.759066641330719
Loss     : 0.3733369670808315

Test:
Batch-1 | Accuracy: 0.835938 | Loss: 0.393824
Batch-2 | Accuracy: 0.843750 | Loss: 0.346045
Batch-3 | Accuracy: 0.882812 | Loss: 0.289405
Batch-4 | Accuracy: 0.875000 | Loss: 0.366903
Accuracy : 0.7615584135055542
Loss     : 0.3490441143512726

EPOCH 12
Train:
Batch-1 | Accuracy: 0.851562 | Loss: 0.321342
Batch-2 | Accuracy: 0.820312 | Loss: 0.415681
Batch-3 | Accuracy: 0.828125 | Loss: 0.426904
Batch-4 | Accuracy: 0.851562 | Loss: 0.307976
Batch-5 | Accuracy: 0.882812 | Loss: 0.305912
Batch-6 | Accuracy: 0.789062 | Loss: 0.481242
Batch-7 | Accuracy: 0.812500 | Loss: 0.486980
Batch-8 | Accuracy: 0.855769 | Loss: 0.348012
Accuracy : 0.7660975456237793
Loss     : 0.386756107211113

Test:
Batch-1 | Accuracy: 0.875000 | Loss: 0.316793
Batch-2 | Accuracy: 0.828125 | Loss: 0.421181
Batch-3 | Accuracy: 0.851562 | Loss: 0.334600
Batch-4 |